In [2]:
# import all library
import os, sys
import numpy as np
import pydub
import librosa
import IPython
import datetime, time
from datetime import datetime
from time import gmtime, strftime
from ADTLib import ADT
import pandas as pd
import copy
import mir_eval
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

print ("### Current Time ###")
print (datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '\n')
print ("### Python Version ###")
print (sys.version + '\n')
print ("### Working Dir ###")
print (os.getcwd())
print ("Basic library load done.")

### Current Time ###
2018-07-26 17:27:41

### Python Version ###
2.7.12 (default, Dec  4 2017, 14:50:18) 
[GCC 5.4.0 20160609]

### Working Dir ###
/host/home/python/ADTLib/ENST_drums_single_song_analysis_v2
Basic library load done.


# Define some functions

In [3]:
# sub-class to sinple-class(KD/SD/HH notes only) transfer function
def get_simple_cls(sub_class):
    # index             0      1      2     3      4     5     6      7      8       9     10    11    12    13
    sub_class_list = ['bd', 'ohh', 'cr1', 'sd', 'chh', 'mt', 'lt', 'ch5', 'rc4', 'spl2', 'rc3', 'cs', 'cb', 'rs']
    
    if sub_class == sub_class_list[0]:          simple_cls = 'KD'
    elif sub_class == sub_class_list[1]:        simple_cls = 'HH'
    elif sub_class == sub_class_list[2]:        simple_cls = 'Others'
    elif sub_class == sub_class_list[3]:        simple_cls = 'SD'
    elif sub_class == sub_class_list[4]:        simple_cls = 'HH'
    elif sub_class == sub_class_list[5]:        simple_cls = 'Others'
    elif sub_class == sub_class_list[6]:        simple_cls = 'Others'
    elif sub_class == sub_class_list[7]:        simple_cls = 'Others'
    elif sub_class == sub_class_list[8]:        simple_cls = 'Others'
    elif sub_class == sub_class_list[9]:        simple_cls = 'Others'
    elif sub_class == sub_class_list[10]:       simple_cls = 'Others'
    elif sub_class == sub_class_list[11]:       simple_cls = 'SD'
    elif sub_class == sub_class_list[12]:       simple_cls = 'Others'
    elif sub_class == sub_class_list[13]:       simple_cls = 'SD'
    else: 
        simple_cls = 'Others'
    
    return simple_cls

# use a class to store file sweep result
class inst_onset_object(object):
    def __init__(self, filename):
        
        # specify processing file name
        self.filename = filename
        
        # specify percussion instruments to evaluate scores
        self.insts = ['KD', 'SD', 'HH', 'Others']
        
        # use inst_act[0] to score KD onset, inst_act[1] to store SD onset, inst_act[2] to store HH onset
        # create emepty list to store result
        self.inst_act = list(range(len(self.insts)))
        for inst in range(len(self.insts)):
            self.inst_act[inst] = []
        
        # store the total number of KD/SD/HH notes
        self.ksh_nnum = 0
        # store the total number of KD/SD/HH/Others notes
        self.total_nnum = 0
        
        
# Define Function to calculate ADT/annotation F-score
def get_fscore(answer_list_in, estimate_list_in):
    # ground gruth note from annotation file
    # flatten 3-instruments KD/SD/HH list into 1-Dim list (MIR-EVAL only accept 1-Dim array)
    answer_list = copy.copy(answer_list_in[0])
    for inst in range(1, len(answer_list_in)):        
        offset_notes = [onset+(500.0*inst) for onset in answer_list_in[inst]]
        answer_list.extend(offset_notes)
    answer_array = np.array(answer_list)

    # estimate note from ADT transcription result
    # flatten 3-instruments KD/SD/HH list into 1-Dim list (MIR-EVAL only accept 1-Dim array)
    estimate_list = copy.copy(estimate_list_in[0])
    for inst in range(1, len(estimate_list_in)):        
        offset_notes = [onset+(500.0*inst) for onset in estimate_list_in[inst]]
        estimate_list.extend(offset_notes)
    estimate_array = np.array(estimate_list)
    
    # Use MIR-eval Library to evaluate f-measure scores
    F_measure, precision ,recall = mir_eval.onset.f_measure(answer_array, estimate_array)
    
    # use a simple list to save result
    all_ground_truth_notes_num = len(answer_list)
    eval_result_list = [all_ground_truth_notes_num, F_measure, precision , recall]
    
    return (eval_result_list)

print ("Define function done.")

Define function done.


# Reload files name (pre-processed list) from  pickle file

In [4]:
fille_name = "enst_mymix_file_namez_list.pkl"

with open(fille_name, "rb") as pkl_file:
    file_names_reload = pickle.load(pkl_file)
    
wav_file_name_list = copy.copy(file_names_reload[1])
txt_file_name_list = copy.copy(file_names_reload[0])

print ("{} wav files are found.".format(len(wav_file_name_list)))
print ("{} corresponding txt files in the list.".format(len(wav_file_name_list)))
print ("wav files name format: {}".format(wav_file_name_list[0]))
print ("txt files name format: {}".format(txt_file_name_list[0]))

58 wav files are found.
58 corresponding txt files in the list.
wav files name format: ../ENST_drums_poly_mymix/merge/output/drmr3_132_minus-one_charleston_sticks_norm.wav
txt files name format: ../ENST_drums_poly_mymix/merge/anno/drmr3_132_minus-one_charleston_sticks.txt


# Loop all files to get F-measure score

In [5]:
# reload pre-processed ADTLib output result (from pickle)
fille_name = "enst_mymix_adt_result_list.pkl"
with open(fille_name, "rb") as pkl_file:
    adt_output_reload = pickle.load(pkl_file)
    
# save singlr file evaluation result into a list
all_file_result_list = []

# loop all files in the ENST dataset (poly-music wet mix files only)
for file_idx in range(0, len(txt_file_name_list)):
    print ("\nStart process File: {}".format(file_idx+1))
    
    # get input file
    file_text = txt_file_name_list[file_idx]
    file_audio = wav_file_name_list[file_idx]
    print (file_text)
    print (file_audio)
    
    
    # Read annotation file by pandas
    file_name = file_text
    file_df = pd.read_csv(file_name, header = None, sep=' ')


    # sweep across rows, if there is any new type of inst for "inst_types", save it into an objest
    row_nums = len(file_df.iloc[:, 1].values)

    # init inst_onset_object
    inst_onset = inst_onset_object(file_name)

    # sweep all rows in the txt file
    for row in range(0, row_nums):
        # get each note's corresponding sub-class instruments & onset time
        inst = str(file_df.iloc[:, 1].values[row]).replace(' ', '')
        act_time = np.float(str(file_df.iloc[:, 0].values[row]))

        # save this note's instrument class and onset time into a single object
        inst_idx = inst_onset.insts.index(get_simple_cls(inst))
        inst_onset.inst_act[inst_idx].append(act_time)

    # calculate kd/sd/hh total notes
    inst_onset.ksh_nnum = len(inst_onset.inst_act[0]) + \
                          len(inst_onset.inst_act[1]) + \
                          len(inst_onset.inst_act[2])


    # do drum transcription
    print ("############    Get Drum Transcription reload data    ############")
    # percussion_out, note_act_data = ADT([file_audio_norm])
    # percussion_out is the original ADTLib output
    # note_act_data is the self-modified note probability function (of time frame)
    percussion_out, note_act_data = (adt_output_reload[file_idx][0], adt_output_reload[file_idx][1])
    kd_note_adt = copy.copy(percussion_out[0]['Kick'])
    sd_note_adt = copy.copy(percussion_out[0]['Snare'])
    hh_note_adt = copy.copy(percussion_out[0]['Hihat'])
    
    # save corresponding file name
    single_song_eval_result = [file_text[:-4].split('/')[-1]]

    # avoid empty list problem
    if len(kd_note_adt)==0:    kd_note_adt_list = []
    else:                      kd_note_adt_list = kd_note_adt.tolist()
    if len(sd_note_adt)==0:    sd_note_adt_list = []
    else:                      sd_note_adt_list = sd_note_adt.tolist()
    if len(hh_note_adt)==0:    hh_note_adt_list = []
    else:                      hh_note_adt_list = hh_note_adt.tolist()
    
    # evaluate all notes scores
    annotation_list = [inst_onset.inst_act[0], inst_onset.inst_act[1], inst_onset.inst_act[2]]
    adt_output_list = [kd_note_adt_list, sd_note_adt_list, hh_note_adt_list]
    eval_result = get_fscore(annotation_list, adt_output_list)
    single_song_eval_result.append(list(['all', eval_result]))

    # evaluate KD only notes scores
    annotation_list = [inst_onset.inst_act[0]]
    adt_output_list = [kd_note_adt_list]
    eval_result = get_fscore(annotation_list, adt_output_list)
    single_song_eval_result.append(list(['kd', eval_result]))

    # evaluate SD only notes scores
    annotation_list = [inst_onset.inst_act[1]]
    adt_output_list = [sd_note_adt_list]
    eval_result = get_fscore(annotation_list, adt_output_list)
    single_song_eval_result.append(list(['sd', eval_result]))

    # evaluate HH only notes scores
    annotation_list = [inst_onset.inst_act[2]]
    adt_output_list = [hh_note_adt_list]
    eval_result = get_fscore(annotation_list, adt_output_list)
    single_song_eval_result.append(list(['hh', eval_result]))
    
    # show processed data scores
    for x in range(0, len(single_song_eval_result)):
        print(single_song_eval_result[x])
        
    # save single file result into a list
    all_file_result_list.append(single_song_eval_result)
    
print ("\n\nAll {} File are evaluated".format(len(txt_file_name_list)))


Start process File: 1
../ENST_drums_poly_mymix/merge/anno/drmr3_132_minus-one_charleston_sticks.txt
../ENST_drums_poly_mymix/merge/output/drmr3_132_minus-one_charleston_sticks_norm.wav
############    Get Drum Transcription reload data    ############
drmr3_132_minus-one_charleston_sticks
['all', [706, 0.5787610619469027, 0.7712264150943396, 0.4631728045325779]]
['kd', [161, 0.396039603960396, 0.975609756097561, 0.2484472049689441]]
['sd', [277, 0.44723618090452255, 0.7355371900826446, 0.3212996389891697]]
['hh', [268, 0.7471698113207547, 0.7557251908396947, 0.7388059701492538]]

Start process File: 2
../ENST_drums_poly_mymix/merge/anno/drmr1_111_minus-one_funky_rods.txt
../ENST_drums_poly_mymix/merge/output/drmr1_111_minus-one_funky_rods_norm.wav
############    Get Drum Transcription reload data    ############
drmr1_111_minus-one_funky_rods
['all', [458, 0.5668449197860963, 0.7310344827586207, 0.462882096069869]]
['kd', [105, 0.018518518518518517, 0.3333333333333333, 0.009523809523

/usr/local/lib/python2.7/dist-packages/mir_eval/onset.py:51: UserWarning: Estimated onsets are empty.
  warnings.warn("Estimated onsets are empty.")


############    Get Drum Transcription reload data    ############
drmr2_150_MIDI-minus-one_soul-98_sticks
['all', [188, 0.6830601092896176, 0.702247191011236, 0.6648936170212766]]
['kd', [52, 0.6976744186046512, 0.8823529411764706, 0.5769230769230769]]
['sd', [31, 0.18604651162790697, 0.3333333333333333, 0.12903225806451613]]
['hh', [105, 0.7679324894514767, 0.6893939393939394, 0.8666666666666667]]

Start process File: 11
../ENST_drums_poly_mymix/merge/anno/drmr1_113_minus-one_charleston_sticks.txt
../ENST_drums_poly_mymix/merge/output/drmr1_113_minus-one_charleston_sticks_norm.wav
############    Get Drum Transcription reload data    ############
drmr1_113_minus-one_charleston_sticks
['all', [410, 0.46115288220551376, 0.4742268041237113, 0.44878048780487806]]
['kd', [37, 0.045454545454545456, 0.14285714285714285, 0.02702702702702703]]
['sd', [141, 0.16923076923076924, 0.18487394957983194, 0.15602836879432624]]
['hh', [232, 0.6518218623481782, 0.6145038167938931, 0.6939655172413793]]


############    Get Drum Transcription reload data    ############
drmr2_116_minus-one_rock-60s_sticks
['all', [355, 0.5134328358208955, 0.546031746031746, 0.48450704225352115]]
['kd', [105, 0.32, 1.0, 0.19047619047619047]]
['sd', [106, 0.3144654088050315, 0.4716981132075472, 0.2358490566037736]]
['hh', [144, 0.6580310880829016, 0.5247933884297521, 0.8819444444444444]]

Start process File: 26
../ENST_drums_poly_mymix/merge/anno/drmr2_134_MIDI-minus-one_country-120_sticks.txt
../ENST_drums_poly_mymix/merge/output/drmr2_134_MIDI-minus-one_country-120_sticks_norm.wav
############    Get Drum Transcription reload data    ############
drmr2_134_MIDI-minus-one_country-120_sticks
['all', [210, 0.7383720930232558, 0.9477611940298507, 0.6047619047619047]]
['kd', [36, 0.2857142857142857, 1.0, 0.16666666666666666]]
['sd', [36, 0.05263157894736842, 0.5, 0.027777777777777776]]
['hh', [138, 0.909090909090909, 0.9523809523809523, 0.8695652173913043]]

Start process File: 27
../ENST_drums_poly_mymix/m

############    Get Drum Transcription reload data    ############
drmr1_137_MIDI-minus-one_rock'n'roll-188_sticks
['all', [306, 0.21296296296296297, 0.36507936507936506, 0.1503267973856209]]
['kd', [150, 0.0, 0.0, 0.0]]
['sd', [123, 0.22068965517241382, 0.7272727272727273, 0.13008130081300814]]
['hh', [33, 0.43795620437956195, 0.28846153846153844, 0.9090909090909091]]

Start process File: 44
../ENST_drums_poly_mymix/merge/anno/drmr1_109_minus-one_metal_sticks.txt
../ENST_drums_poly_mymix/merge/output/drmr1_109_minus-one_metal_sticks_norm.wav
############    Get Drum Transcription reload data    ############
drmr1_109_minus-one_metal_sticks
['all', [430, 0.472972972972973, 0.8641975308641975, 0.32558139534883723]]
['kd', [131, 0.0, 0.0, 0.0]]
['sd', [77, 0.025316455696202535, 0.5, 0.012987012987012988]]
['hh', [222, 0.7296587926509187, 0.8742138364779874, 0.6261261261261262]]

Start process File: 45
../ENST_drums_poly_mymix/merge/anno/drmr2_142_MIDI-minus-one_nu-soul_sticks.txt
../ENST

# Get dataset Avg. F-measure score

In [6]:
total_onset_notes = 0
note_times_score_fscr = 0
note_times_score_prec = 0
note_times_score_recl = 0

for file_idx in range(0, len(all_file_result_list)):
    total_onset_notes += all_file_result_list[file_idx][1][1][0]
    note_times_score_fscr += all_file_result_list[file_idx][1][1][0] * all_file_result_list[file_idx][1][1][1]
    note_times_score_prec += all_file_result_list[file_idx][1][1][0] * all_file_result_list[file_idx][1][1][2]
    note_times_score_recl += all_file_result_list[file_idx][1][1][0] * all_file_result_list[file_idx][1][1][3]
    
avg_fscr = note_times_score_fscr / total_onset_notes
avg_prec = note_times_score_prec / total_onset_notes
avg_recl = note_times_score_recl / total_onset_notes

print ("onset notes:  {}".format(total_onset_notes))
print ("F-Measure:    {:.3f}".format(avg_fscr))
print ("Precision:    {:.3f}".format(avg_prec))
print ("Recall:       {:.3f}".format(avg_recl))

onset notes:  20458
F-Measure:    0.473
Precision:    0.675
Recall:       0.373


# Show sorted Top-N score result

In [7]:
top_n = 5
#top_n = len(all_file_result_list_reload)

# sort files by f-measure score (High to low)
all_fscores = []
for file_idx in range(0, len(all_file_result_list)):
    all_fscores.append(all_file_result_list[file_idx][1][1][1])   
rank_list = []
for file_idx in range(0, len(all_file_result_list)):
    rank_list.append(all_fscores.index(np.sort(all_fscores)[::-1][file_idx]))
    
# show files by f-measure score (High to low)
for file_idx in range(0, top_n):
    print("File name: {}".format(all_file_result_list[rank_list[file_idx]][0]))
# show notes by f-measure score (High to low)
for file_idx in range(0, top_n):
    print("Notes: {}".format(all_file_result_list[rank_list[file_idx]][1][1][0]))
# show f-measure scores (High to low)
for file_idx in range(0, top_n):
    print("F-Measure: {:.3f}".format(all_file_result_list[rank_list[file_idx]][1][1][1]))

File name: drmr2_134_MIDI-minus-one_country-120_sticks
File name: drmr1_139_MIDI-minus-one_soul-120-marvin-gaye_sticks
File name: drmr2_142_MIDI-minus-one_nu-soul_sticks
File name: drmr2_148_MIDI-minus-one_soul-120-marvin-gaye_sticks
File name: drmr2_150_MIDI-minus-one_soul-98_sticks
Notes: 210
Notes: 192
Notes: 278
Notes: 218
Notes: 188
F-Measure: 0.738
F-Measure: 0.712
F-Measure: 0.690
F-Measure: 0.689
F-Measure: 0.683
